# Lexical Baseline

In [1]:
from google.colab import drive
ROOT = '/content/drive'
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import sys
from os.path import join 
parent_dr = os.path.split(os.getcwd())[0]
if parent_dr not in sys.path:
    sys.path.append(parent_dr)

In [3]:
repo_dir = '/content/drive/MyDrive/nlp_group_proj/metaphor-detection'
if repo_dir not in sys.path:
    sys.path.append(repo_dir)
#join(repo_dir)

In [22]:
import numpy as np
import random
from core.data.gao_data import *
from models.classification import lexical_baseline as lb

In [5]:
pd.set_option('display.max_colwidth', None)

In [6]:
# gao et all data directory
data_dir = os.path.join("resources", "metaphor-in-context", "data")
print(f"Data directory: {data_dir}")

Data directory: resources/metaphor-in-context/data


In [7]:
data_container = ExperimentData(data_dir)

In [8]:
PROJECT_PATH = join(repo_dir)
%cd "{PROJECT_PATH}"

/content/drive/MyDrive/nlp_group_proj/metaphor-detection


In [9]:
%pwd

'/content/drive/MyDrive/nlp_group_proj/metaphor-detection'

**Read in all data**

In [10]:
data_container.read_all_data(to_pandas=False)

MOH formatted nrow: 1603
MOH train nrow: 1283
MOH test nrow: 160
MOH val nrow: 160
MOH-X formatted svo nrow: 647
MOH-X formatted svo cleaned nrow: 647
TroFi formatted_all3737 nrow: 3737
TroFi-X formatted svo nrow: 1444
VUA formatted nrow: 23113
VUA train nrow: 15516
VUA train augmented nrow: 116622
VUA train no val nrow: 12541
VUA test nrow: 5873
VUA val nrow: 1724
VUA_seq train nrow: 6323
VUA_seq test nrow: 2694
VUA_seq val nrow: 1550


In [11]:
print(len(data_container.vua_formatted_train))
print(len(data_container.vua_seq_formatted_test))

15516
2694


In [14]:
tupled_vua_train = lb.process_vua_to_tuple(data_container.vua_formatted_train)
tupled_vua_test = lb.process_vua_to_tuple(data_container.vua_formatted_test)

In [16]:
vua_LB_model = lb.LexicalBaseline()

Initializing new Lexical Baseline model


In [17]:
vua_LB_model.create_CLS_Model(tupled_vua_train)

In [18]:
vua_LB_model.CLS_predict(tupled_vua_test)

None


In [19]:
print(len(vua_LB_model.predictions))
print(len(vua_LB_model.labels))
vua_LB_model.evaluate()

5873
5873

                Accuracy: 76.24723310063
                Precision: 68.22709163346613
                Recall: 38.898353208404316
                F1: 49.54792043399638
                


Cross Fold Validaton

In [20]:
len(data_container.moh_x_formatted_svo)

647

In [33]:
tupled_moh_x_train = lb.process_moh_x_to_tuple(data_container.moh_x_formatted_svo)
len(tupled_moh_x_train)

647

In [34]:
# upside down floor division
lines_per_fold = -(len(tupled_moh_x_train) // -10)
print(lines_per_fold)

65


In [38]:
# setting same seed as Gao et al.
random.seed(3)
random.shuffle(tupled_moh_x_train)

# prepare 10 folds
ten_folds = []
for i in range(10):
    ten_folds.append(tupled_moh_x_train[i*lines_per_fold: (i+1)*lines_per_fold])

print(len(ten_folds))
# 10 fold
PRFA_list = []
for i in range(10):
    raw_train_mohX = []
    raw_val_mohX = []
    # separate training and validation data
    for j in range(10):
        if j != i:
            raw_train_mohX.extend(ten_folds[j])
        else:
            raw_val_mohX = ten_folds[j]
    # make model, predict, and evaluate
    mohx_model = lb.LexicalBaseline()
    mohx_model.create_CLS_Model(raw_train_mohX)
    mohx_model.CLS_predict(raw_val_mohX)
    mohx_model.evaluate()
      

10
Initializing new Lexical Baseline model

                Accuracy: 41.53846153846154
                Precision: 33.333333333333336
                Recall: 22.580645161290324
                F1: 26.923076923076923
                
Initializing new Lexical Baseline model

                Accuracy: 43.07692307692308
                Precision: 33.333333333333336
                Recall: 23.333333333333332
                F1: 27.45098039215686
                
Initializing new Lexical Baseline model

                Accuracy: 46.15384615384615
                Precision: 30.0
                Recall: 22.22222222222222
                F1: 25.53191489361702
                
Initializing new Lexical Baseline model

                Accuracy: 40.0
                Precision: 46.666666666666664
                Recall: 18.42105263157895
                F1: 26.41509433962264
                
Initializing new Lexical Baseline model

                Accuracy: 41.53846153846154
                Precisio